In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm 

In [2]:
delta_df = pd.read_csv('../data/processed/top10_speed_results.csv')
delta_df

,driverId,driverRef,number,code,dob,nationality,full name,Slowest Lap Time,max_x,Average Lap Time,Minimum Speed,Maximum Speed,Average Speed,Minimum Laps,Maximum Laps,Average Laps
0,1,hamilton,44,HAM,1985-01-07,British,Lewis Hamilton,0.000,119.560,89.571771,0.000,255.014,201.224538,0,76,43.332061
1,842,gasly,10,GAS,1996-02-07,French,Pierre Gasly,0.000,109.700,84.602356,0.000,248.611,198.936186,0,72,45.915254
2,817,ricciardo,3,RIC,1989-07-01,Australian,Daniel Ricciardo,0.000,119.831,90.965011,0.000,250.174,199.633481,0,78,45.360656
3,822,bottas,77,BOT,1989-08-28,Finnish,Valtteri Bottas,0.000,151.939,90.828822,0.000,251.690,203.473487,0,74,47.019737
4,830,max_verstappen,33,VER,1997-09-30,Dutch,Max Verstappen,0.000,140.776,85.783746,0.000,250.830,195.509430,0,70,43.035088
5,832,sainz,55,SAI,1994-09-01,Spanish,Carlos Sainz,0.000,134.103,84.740167,0.000,249.671,188.966746,0,70,41.035088
6,844,leclerc,16,LEC,1997-10-16,Monegasque,Charles Leclerc,0.000,107.840,85.467019,0.000,251.235,202.433019,0,70,43.259259
7,846,norris,4,NOR,1999-11-13,British,Lando Norris,67.193,108.552,88.388000,157.214,248.141,212.342030,4,71,45.272727
8,848,albon,23,ALB,1996-03-23,Thai,Alexander Albon,67.299,108.736,87.662303,158.890,250.165,214.058455,9,68,48.030303


In [3]:
delta_df['Driver 1'] = delta_df['number']
delta_df['Driver 1 Average Lap Time'] = delta_df['Average Lap Time']
delta_df.rename(columns = {'number':'Driver 2'}, inplace = True)
delta_df.rename(columns = {'Average Lap Time':'Driver 2 Average Lap Time'}, inplace = True)
delta_df

,driverId,driverRef,Driver 2,code,dob,nationality,full name,Slowest Lap Time,max_x,Driver 2 Average Lap Time,Minimum Speed,Maximum Speed,Average Speed,Minimum Laps,Maximum Laps,Average Laps,Driver 1,Driver 1 Average Lap Time
0,1,hamilton,44,HAM,1985-01-07,British,Lewis Hamilton,0.000,119.560,89.571771,0.000,255.014,201.224538,0,76,43.332061,44,89.571771
1,842,gasly,10,GAS,1996-02-07,French,Pierre Gasly,0.000,109.700,84.602356,0.000,248.611,198.936186,0,72,45.915254,10,84.602356
2,817,ricciardo,3,RIC,1989-07-01,Australian,Daniel Ricciardo,0.000,119.831,90.965011,0.000,250.174,199.633481,0,78,45.360656,3,90.965011
3,822,bottas,77,BOT,1989-08-28,Finnish,Valtteri Bottas,0.000,151.939,90.828822,0.000,251.690,203.473487,0,74,47.019737,77,90.828822
4,830,max_verstappen,33,VER,1997-09-30,Dutch,Max Verstappen,0.000,140.776,85.783746,0.000,250.830,195.509430,0,70,43.035088,33,85.783746
5,832,sainz,55,SAI,1994-09-01,Spanish,Carlos Sainz,0.000,134.103,84.740167,0.000,249.671,188.966746,0,70,41.035088,55,84.740167
6,844,leclerc,16,LEC,1997-10-16,Monegasque,Charles Leclerc,0.000,107.840,85.467019,0.000,251.235,202.433019,0,70,43.259259,16,85.467019
7,846,norris,4,NOR,1999-11-13,British,Lando Norris,67.193,108.552,88.388000,157.214,248.141,212.342030,4,71,45.272727,4,88.388000
8,848,albon,23,ALB,1996-03-23,Thai,Alexander Albon,67.299,108.736,87.662303,158.890,250.165,214.058455,9,68,48.030303,23,87.662303


In [8]:
matrix_df = delta_df.loc[:,['Driver 1', 'Driver 1 Average Lap Time', 'Driver 2', 'Driver 2 Average Lap Time']]
matrix_df

,Driver 1,Driver 1 Average Lap Time,Driver 2,Driver 2 Average Lap Time
0,44,89.571771,44,89.571771
1,10,84.602356,10,84.602356
2,3,90.965011,3,90.965011
3,77,90.828822,77,90.828822
4,33,85.783746,33,85.783746
5,55,84.740167,55,84.740167
6,16,85.467019,16,85.467019
7,4,88.388000,4,88.388000
8,23,87.662303,23,87.662303


In [10]:
rank_matrix = pd.DataFrame([[1, 822, 43.332061, 90], 
                            [50, 1, 87, 43.332061],
                            [2, 0, 87, 90],
                            [3, 4, 88, 92],
                            [3, 1, 88, 90],
                            [1, 4, 90, 92], 
                            [1, 4, 90, 92]
                            [1, 4, 90, 92]], 
                           columns=['Driver1', 'Driver2', 'Driver1_laptime', 'Driver2_laptime']